In [ ]:
!pip install langchain==0.3.0 langchain-openai==0.2.0 langgraph==0.2.22 httpx==0.27.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB

In [ ]:
import operator
from typing import Annotated, Any, Optional

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.graph import END, StateGraph
from pydantic import BaseModel, Field

import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-book"


# ペルソナを表すデータモデル
class Persona(BaseModel):
    name: str = Field(..., description="ペルソナの名前")
    background: str = Field(..., description="ペルソナの持つ背景")


# ペルソナのリストを表すデータモデル
class Personas(BaseModel):
    personas: list[Persona] = Field(
        default_factory=list, description="ペルソナのリスト"
    )


# インタビュー内容を表すデータモデル
class Interview(BaseModel):
    persona: Persona = Field(..., description="インタビュー対象のペルソナ")
    question: str = Field(..., description="インタビューでの質問")
    answer: str = Field(..., description="インタビューでの回答")


# インタビュー結果のリストを表すデータモデル
class InterviewResult(BaseModel):
    interviews: list[Interview] = Field(
        default_factory=list, description="インタビュー結果のリスト"
    )


# （情報評価用の EvaluationResult クラスは削除）


# 要件定義生成AIエージェントのステート
class InterviewState(BaseModel):
    user_request: str = Field(..., description="ユーザーからのリクエスト")
    personas: Annotated[list[Persona], operator.add] = Field(
        default_factory=list, description="生成されたペルソナのリスト"
    )
    interviews: Annotated[list[Interview], operator.add] = Field(
        default_factory=list, description="実施されたインタビューのリスト"
    )
    requirements_doc: str = Field(default="", description="生成された要件定義")
    iteration: int = Field(
        default=0, description="ペルソナ生成とインタビューの反復回数"
    )
    # ↓ 情報評価ステップ削除に伴い、このフラグも削除
    # is_information_sufficient: bool = Field(
    #     default=False, description="情報が十分かどうか"
    # )


# ペルソナを生成するクラス
class PersonaGenerator:
    def __init__(self, llm: ChatOpenAI, k: int = 5):
        self.llm = llm.with_structured_output(Personas)
        self.k = k

    def run(self, user_request: str) -> Personas:
        # プロンプトテンプレートを定義
        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "あなたはSNSアカウントのターゲットユーザーへのインタビュー用の多様なペルソナを作成する専門家です。",
                ),
                (
                    "human",
                    f"以下のSNSのトピックに関するインタビュー用に、{self.k}人の多様なペルソナを生成してください。\n\n"
                    "トピック: {user_request}\n\n"
                    "各読者ペルソナには名前と簡単な背景を含めてください。年齢、性別、職業、トピックに対する知識レベルにおいて多様性を確保してください。",
                ),
            ]
        )
        chain = prompt | self.llm
        return chain.invoke({"user_request": user_request})


# インタビューを実施するクラス
class InterviewConductor:
    def __init__(self, llm: ChatOpenAI):
        self.llm = llm

    def run(self, user_request: str, personas: list[Persona]) -> InterviewResult:
        # 質問を生成
        questions = self._generate_questions(
            user_request=user_request, personas=personas
        )
        # 回答を生成
        answers = self._generate_answers(personas=personas, questions=questions)
        # 組み合わせからインタビューリストを作成
        interviews = self._create_interviews(personas, questions, answers)
        return InterviewResult(interviews=interviews)

    def _generate_questions(
        self, user_request: str, personas: list[Persona]
    ) -> list[str]:
        question_prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "あなたはインタビュアーです。ペルソナの悩みや課題を引き出すための質問を作成します。",
                ),
                (
                    "human",
                    "以下の読者ペルソナが、ブログ記事のトピックに関して自身の悩みや課題を話すための、オープンな質問を一つ作成してください。\n\n"
                    "トピック: {user_request}\n"
                    "読者ペルソナ: {persona_name} - {persona_background}\n\n"
                    "質問はシンプルで、このペルソナが自分の悩みを率直に話せるようにしてください。",
                ),
            ]
        )
        question_chain = question_prompt | self.llm | StrOutputParser()
        question_queries = [
            {
                "user_request": user_request,
                "persona_name": persona.name,
                "persona_background": persona.background,
            }
            for persona in personas
        ]
        return question_chain.batch(question_queries)

    def _generate_answers(
        self, personas: list[Persona], questions: list[str]
    ) -> list[str]:
        answer_prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "あなたは以下の読者ペルソナです。インタビュアーの質問に対して、あなたが抱えている悩みや課題を具体的に教えてください。\n\nペルソナ:  {persona_name} - {persona_background}",
                ),
                ("human", "質問: {question}"),
            ]
        )
        answer_chain = answer_prompt | self.llm | StrOutputParser()
        answer_queries = [
            {
                "persona_name": persona.name,
                "persona_background": persona.background,
                "question": question,
            }
            for persona, question in zip(personas, questions)
        ]
        return answer_chain.batch(answer_queries)

    def _create_interviews(
        self, personas: list[Persona], questions: list[str], answers: list[str]
    ) -> list[Interview]:
        return [
            Interview(persona=persona, question=q, answer=a)
            for persona, q, a in zip(personas, questions, answers)
        ]


# （情報評価クラス InformationEvaluator は削除）


# 要件定義書を生成するクラス
class RequirementsDocumentGenerator:
    def __init__(self, llm: ChatOpenAI):
        self.llm = llm

    def run(self, user_request: str, interviews: list[Interview]) -> str:
        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "あなたは収集した情報に基づいてSNSアカウントの運用マニュアルを作成する専門家です。",
                ),
                (
                    "human",
                    "以下のトピックと複数の読者ペルソナからのインタビュー結果に基づいて、SNSアカウントの運用マニュアルの指示書を作成してください。\n\n"
                    "トピック: {user_request}\n\n"
                    "インタビュー結果:\n{interview_results}\n"
                    "運用マニュアルには以下のセクションを含めてください:\n"
                    "1. SNS運用の目的\n"
                    "2. ターゲット読者\n"
                    "3. ターゲット読者の悩み\n"
                    "4. トピックとハッシュタグ\n"
                    "5. 運用するSNSプラットフォームと投稿頻度、タイミング、投稿形式（テキスト、画像、動画、ストーリー、ライブ配信など）\n"
                    "6. 投稿内容のテーマ\n"
                    "7. 注意事項\n"
                    "出力は必ず日本語でお願いします。\n\n記事作成の指示書:",
                ),
            ]
        )
        chain = prompt | self.llm | StrOutputParser()

        # インタビュー結果をテキスト形式にまとめる
        interview_results_text = "\n".join(
            f"ペルソナ: {i.persona.name} - {i.persona.background}\n"
            f"質問: {i.question}\n回答: {i.answer}\n"
            for i in interviews
        )

        return chain.invoke(
            {
                "user_request": user_request,
                "interview_results": interview_results_text,
            }
        )


# 要件定義書生成AIエージェントのクラス
class DocumentationAgent:
    def __init__(self, llm: ChatOpenAI, k: Optional[int] = None):
        self.persona_generator = PersonaGenerator(llm=llm, k=k)
        self.interview_conductor = InterviewConductor(llm=llm)
        # ↓ 情報評価クラスを削除したのでここも削除
        # self.information_evaluator = InformationEvaluator(llm=llm)
        self.requirements_generator = RequirementsDocumentGenerator(llm=llm)
        self.graph = self._create_graph()

    def _create_graph(self) -> StateGraph:
        workflow = StateGraph(InterviewState)

        # ノード追加（情報評価工程は入れない）
        workflow.add_node("generate_personas", self._generate_personas)
        workflow.add_node("conduct_interviews", self._conduct_interviews)
        workflow.add_node("generate_requirements", self._generate_requirements)

        # エントリーポイント
        workflow.set_entry_point("generate_personas")

        # 遷移設定
        workflow.add_edge("generate_personas", "conduct_interviews")
        # 評価をスキップし直接要件定義書生成へ
        workflow.add_edge("conduct_interviews", "generate_requirements")

        workflow.add_edge("generate_requirements", END)

        return workflow.compile()

    def _generate_personas(self, state: InterviewState) -> dict[str, Any]:
        new_personas: Personas = self.persona_generator.run(state.user_request)
        return {
            "personas": new_personas.personas,
            "iteration": state.iteration + 1,
        }

    def _conduct_interviews(self, state: InterviewState) -> dict[str, Any]:
        # ペルソナが多い場合は最後の5人のみに絞る
        new_personas = state.personas[-5:]
        new_interviews: InterviewResult = self.interview_conductor.run(
            state.user_request, new_personas
        )
        return {"interviews": new_interviews.interviews}

    # ↓ 情報評価は削除
    # def _evaluate_information(self, state: InterviewState) -> dict[str, Any]:
    #     ...

    def _generate_requirements(self, state: InterviewState) -> dict[str, Any]:
        requirements_doc: str = self.requirements_generator.run(
            state.user_request, state.interviews
        )
        return {"requirements_doc": requirements_doc}

    def run(self, user_request: str) -> str:
        initial_state = InterviewState(user_request=user_request)
        final_state = self.graph.invoke(initial_state)
        return final_state["requirements_doc"]


# メイン関数
def main():
    user_request = input("運用したいSNSアカウントについて記載してください: ")
    k = 3  # ペルソナの人数（必要に応じて変更可能）

    llm = ChatOpenAI(model_name="gpt-4o-2024-11-20", temperature=0.0)
    # llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0)

    agent = DocumentationAgent(llm=llm, k=k)
    final_output = agent.run(user_request=user_request)

    print(final_output)


if __name__ == "__main__":
    main()


運用したいSNSアカウントについて記載してください: 副業ですが、東京でSEOや広告、デジタルマーケティングの業務支援サービスを販売をしたいです。 Python、データサイエンス、AI開発が得意なので、その強みを活かしたいです。あと英語ができるので、東京にいる外国人相手にビジネスがしたいです。
### SNSアカウント運用マニュアル

---

#### 1. SNS運用の目的
- **副業としてのデジタルマーケティング業務支援サービスの認知拡大と顧客獲得**
  - SEO、広告運用、データサイエンス、AI開発のスキルを活かし、東京を拠点に日本人および外国人向けにサービスを提供。
  - ターゲット読者の悩みを解決する情報を発信し、信頼を構築。
  - 自身の専門性をアピールし、クライアント獲得につなげる。

---

#### 2. ターゲット読者
1. **田中翔太（32歳、日本人男性）**
   - 中小企業のマーケティング担当者。
   - SEOや広告の基礎知識はあるが、データサイエンスやAIは初心者。
   - 副業に興味があり、スキルアップを目指している。

2. **エミリー・ジョンソン（28歳、アメリカ人女性）**
   - 東京在住の英語教師。
   - デジタルマーケティングやPython、AIに興味を持ち始めた初心者。
   - 英語を活かして副業を始めたいと考えている。

3. **佐藤美咲（40歳、日本人女性）**
   - フリーランスのデータアナリスト。
   - データサイエンスやPythonに精通しているが、SEOや広告の知識は浅い。
   - 外国人向けビジネスに挑戦したいと考えている。

---

#### 3. ターゲット読者の悩み
- **田中翔太**
  - 自分のスキルを副業にどう結びつけるか分からない。
  - データサイエンスやAIの学習方法が分からない。
  - クライアント獲得や営業に不安がある。

- **エミリー・ジョンソン**
  - デジタルマーケティングやPython、AIの初心者で、どこから始めればいいか分からない。
  - 英語を活かした副業の具体的なイメージが湧かない。
  - 限られた時間で効率的に学びたい。

- **佐藤美咲**
  - SEOや広告の基礎知識が不足している。
  - 外